In [100]:
# отключим всякие предупреждения Anaconda
import warnings

warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, LassoCV, LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import (GridSearchCV, cross_val_score,
                                     train_test_split)
from sklearn.preprocessing import StandardScaler

**Будем работать с набором данных по качеству белого вина (репозиторий UCI).**
**Загружаем данные.**

In [101]:
data = pd.read_csv('./data/winequality-white.csv', sep=';')

In [102]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [103]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


**Отделите целевой признак, разделите обучающую выборку в отношении 7:3 (30% - под оставленную выборку, пусть random_state=17) и отмасштабируйте данные с помощью StandardScaler.**

In [104]:
y = data['quality']
x = data.drop(columns=["quality"])

In [105]:
X_train, X_holdout, y_train, y_holdout = train_test_split(x, y, test_size=0.3, random_state=10) # Ваш код здесь
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)

## Линейная регрессия

**Обучите простую линейную регрессию.**

In [106]:
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)
y_pred = linreg.predict(X_holdout_scaled)
y_train_pred = linreg.predict(X_train_scaled)

**<font color='red'>Вопрос 1:</font> Каковы среднеквадратичные ошибки линейной регрессии на обучающей и отложенной выборках?**

In [119]:
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, y_pred, squared=False))
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, y_train_pred, squared=False))

Mean squared error (test): 0.552
Mean squared error (train): 0.755


**Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина (учтите, что большие по модулю отрицательные значения коэффициентов тоже говорят о сильном влиянии). Создайте для этого новый небольшой DataFrame.**<br>
**<font color='red'>Вопрос 2:</font> Какой признак линейная регрессия считает наиболее сильно влияющим на качество вина?**

In [108]:
numeric_data = X_train.select_dtypes([np.number])
linreg_coef = pd.DataFrame(linreg.coef_, index=numeric_features)
linreg_coef.sort_values(0)

,0
density,-0.419620
volatile acidity,-0.192393
citric acid,0.000092
chlorides,0.002504
total sulfur dioxide,0.005986
fixed acidity,0.033485
free sulfur dioxide,0.072698
sulphates,0.080609
pH,0.089232
alcohol,0.266679


## Lasso-регрессия

**Обучите Lasso-регрессию с небольшим коэффициентом $\alpha = 0.01$ (слабая регуляризация). Пусть опять random_state=17.**

In [117]:
lasso1 = Lasso(alpha = 0.01)
lasso1.fit(X_train_scaled, y_train)
y_pred_lasso = lasso1.predict(X_holdout_scaled)
y_train_pred_lasso = lasso1.predict(X_train_scaled)

**Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина. Какой признак "отвалился" первым, то есть наименее важен для объяснения целевого признака в модели Lasso?**

In [110]:
lasso1_coef = pd.DataFrame(lasso1.coef_, index=numeric_features)
lasso1_coef.sort_values(0)
#Ответ total sulfur dioxide

,0
volatile acidity,-0.189138
density,-0.144007
fixed acidity,-0.019539
chlorides,-0.003708
citric acid,-0.000000
total sulfur dioxide,-0.000000
pH,0.034417
sulphates,0.055409
free sulfur dioxide,0.072379
residual sugar,0.199437


**Теперь определите лучшее значение $\alpha$ в процессе кросс-валидации 5-кратной кросс-валидации. Используйте LassoCV и random_state=17.**

In [111]:
alphas = np.logspace(-6, 2, 200)
#lasso_cv = GridSearchCV(Lasso(), [{"alpha": alphas}], scoring="neg_root_mean_squared_error", cv=5)
lasso_cv = LassoCV(cv=5, n_jobs=-1)
lasso_cv.fit(X_train_scaled, y_train)

LassoCV(cv=5, n_jobs=-1)

In [112]:
lasso_cv.alpha_

0.004707754663988844

**Выведите коэффициенты "лучшего" Lasso в порядке убывания влияния на качество вина. **<br>
**<font color='red'>Вопрос 3:</font> Какой признак "обнулился первым" в настроенной модели LASSO?**

In [113]:
lasso_cv_coef = pd.DataFrame(lasso_cv.coef_, index=numeric_features)
lasso_cv_coef.sort_values(0)

,0
density,-0.269241
volatile acidity,-0.191236
chlorides,-0.001337
fixed acidity,-0.000000
citric acid,-0.000000
total sulfur dioxide,0.000000
pH,0.058244
sulphates,0.067968
free sulfur dioxide,0.074165
residual sugar,0.290796


**Оцените среднеквадратичную ошибку модели на обучающей и тестовой выборках.**<br>
**<font color='red'>Вопрос 4:</font> Каковы среднеквадратичные ошибки настроенной LASSO-регрессии на обучающей и отложенной выборках?**

In [121]:
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, y_pred_lasso, squared=False))
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, y_train_pred_lasso, squared=False))

Mean squared error (test): 0.745
Mean squared error (train): 0.757
